In [1]:
import requests
from bs4 import BeautifulSoup
import os

In [2]:
def scrape_paper(url, save_folder):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h2', class_='subtitle').text.strip()
    abstract = soup.find('p', class_='abstract').text.strip()
    content = soup.find('pre').text.strip()

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    filename = f"{save_folder}/{title}.txt"
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Title: {title}\n\n")
        file.write(f"Abstract: {abstract}\n\n")
        file.write(f"Content:\n{content}")

    print(f"Saved: {filename}")

In [9]:
base_url = "https://proceedings.neurips.cc/paper_files/paper/2023" ## TO BE CHANGED FOR ICLR
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')
paper_links = soup.find_all('a', class_='conference') ## TO BE CHANGED FOR ICLR
print(paper_links)

[]


In [10]:
## TO BE CHANGED FOR ICLR

for link in paper_links:
    print(link)
    paper_url = base_url + "/" + link['href'].replace("-Abstract-", "-Paper-")
    scrape_paper(paper_url, "../dataset/NeurIPS_2023_Papers")